# Homework 3

## FINM 37400 - 2024

### UChicago Financial Mathematics

* Raul Renteria
* rrenteria@uchicago.edu

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per \\$1 face value
* Class terminology: Modified dollar duration is .1746 per \\$100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per \\$100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount \\$0.97 billion**
    
    No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

In [23]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
from scipy.optimize import fsolve

import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13
import warnings
warnings.filterwarnings('ignore')

***

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [68]:
import pandas as pd


def calculate_bond_price(coupon_rate, years_to_maturity, yield_to_maturity, face_value=100, compounding_frequency=2):
    """
    Calculate the price of a bond with custom compounding frequency.

    Parameters:
    - face_value: The face value of the bond.
    - coupon_rate: The annual coupon rate (as a percentage).
    - years_to_maturity: The number of years to maturity.
    - yield_to_maturity: The yield to maturity (as a percentage).
    - compounding_frequency: The number of compounding periods per year.

    Returns:
    The calculated price of the bond.
    """
    #coupon_rate /= 100
    #yield_to_maturity /= 100

    periodic_coupon_rate = coupon_rate / compounding_frequency
    periodic_yield = yield_to_maturity / compounding_frequency
    periods = int(years_to_maturity * compounding_frequency)

    pv_coupons = 0
    for period in range(1, periods + 1):
        pv_coupons += face_value * periodic_coupon_rate  / (1 + periodic_yield)**period

    present_value_face_value = face_value / (1 + periodic_yield)**periods

    bond_price = pv_coupons + present_value_face_value

    return bond_price

face_value = 100
coupon_rate = 0.045
years_to_maturity = 30
yield_to_maturity = 0.04193
swap_rate = 0.04256

bond_price = calculate_bond_price(coupon_rate, years_to_maturity, yield_to_maturity)
print(f"The new price of the bond is: ${bond_price:.2f}")

The new price of the bond is: $105.21


## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

In [85]:
cash_flows = pd.DataFrame(index=['2009-05-04'])
cash_flows['T-bond'] = face_value * coupon_rate / 2
cash_flows['Swap'] = -face_value * swap_rate / 2
cash_flows['Total'] = cash_flows.sum(axis=1)
cash_flows

,T-bond,Swap,Total
2009-05-04,2.25,-2.128,0.122


Since the floating rate of the swap equals the repo rate used in the trade, these just cancel so we don't include them in the projected cashflows

## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [58]:
def calculate_duration(tau, ytm, coupon_rate, freq=2):
    y_tilde = ytm / freq
    c_tilde = coupon_rate / freq
    tau_tilde = tau * freq
        
    if coupon_rate == ytm:
        duration = (1 + y_tilde) / y_tilde  * (1 - 1 / (1 + y_tilde)**tau_tilde)
        
    else:
        duration = (1 + y_tilde) / y_tilde - (1 + y_tilde + tau_tilde *(c_tilde - y_tilde)) / (c_tilde * ((1 + y_tilde)**tau_tilde - 1) + y_tilde)

    duration *= 1 / freq
    
    return duration

bond_duration = calculate_duration(30, yield_to_maturity, coupon_rate)
fixed_rate_bond = calculate_duration(30, swap_rate, swap_rate)
frn = 0.5
swap_duration = fixed_rate_bond - frn

durations = pd.DataFrame({'Macauley duration' : [bond_duration, swap_duration],
                          'Dollar duration ($)' : [bond_duration * bond_price, swap_duration * face_value]},
                         index=['T-bond', 'Swap'])
durations

,Macauley duration,Dollar duration ($)
T-bond,17.083633,1797.425100
Swap,16.712744,1671.274445


The duaration of the swap is positive, and it's smaller in magnitude than the T-bond

## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

Using the equation from homework 2:
$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

In [96]:
notional = 500e6
hedge_ratio = (swap_duration * face_value) / (bond_duration * bond_price)
bond_position_size = (notional * hedge_ratio) / bond_price
print(f' position size on the bond: {round(bond_position_size)}') 

 position size on the bond: 4418719


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

- The rising YTM means the price of the bond decreases. This effects your funding as you used repo to fund this position with the 30 year T-bond as collateral.
- Drop in the swap rate means LIBOR rates have gone down  

500e6## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion 1 notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$\begin{align*}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align*}
$

In [176]:
prices = pd.DataFrame(index=['2009-05-04'])
prices['T-bond Price ($)'] = calculate_bond_price(coupon_rate, 29.5, 0.0436)
prices['Swap Price ($)'] = calculate_bond_price(swap_rate, 29.5, 0.0408) - face_value
prices

,T-bond Price ($),Swap Price ($)
2009-05-04,102.311401,3.003349


## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [203]:
pnl_summary = pd.DataFrame(index=[['T-bond', 'Swap']], dtype=float)
pnl_summary['Cash flows'] = (cash_flows[['T-bond', 'Swap']].values * [bond_position_size, -(notional / 100)]).T
#pnl_summary['Position Values'] = ((prices.values - d.values)[0] * [bond_position_size, -(500e6 / 100)])
pnl_summary.loc['Total'] = pnl_summary.sum()
pnl_summary['PnL'] = pnl_summary.sum(axis=1)
pnl_summary

,Cash flows,Position Values,PnL
"(T-bond,)",9.942118e+06,-1.282260e+07,-2.880479e+06
"(Swap,)",1.064000e+07,-1.501675e+07,-4.376747e+06
Total,2.058212e+07,-2.783934e+07,-7.257226e+06


In [197]:
d = pd.DataFrame(columns=prices.columns)
d.loc['2008-11-04'] = [bond_price, 0]
(d.values - prices.values)[0] 

array([ 2.90188107, -3.00334941])

***